In [1]:
import os
import glob
from pathlib import Path
import yaml

import json
import logging
import wandb
from datetime import datetime, timezone

import pandas as pd 
import numpy as np 

# Custom Utilities Module
from utils.paths import get_paths
from utils.file_io import ingest_data, save_data
from utils.logging_setup import configure_logging
from utils.eda_logging import profile_dataframe

'''
from utils.wandb_utils import (
    log_metrics,
    log_dataframe_head,
    log_dir_as_artifact,
    log_files_as_artifact,
)
'''
from utils.wandb_utils import finalize_wandb_stage


# Show more columns
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)



In [2]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [3]:
def log_bronze_paths(paths, logger: logging.Logger) -> None:
    logger.info("Project Root Path Loaded: %s", paths.root)
    logger.info("Project Logging Path Loaded: %s", paths.logs)
    logger.info("Project Artifacts Path Loaded: %s", paths.artifacts)
    logger.info("Project Notebooks Path Loaded %s", paths.notebooks)
    logger.info("Project Raw Path Loaded: %s", paths.data_raw)

    logger.info("Project Data Path Loaded: %s", paths.data)

    logger.info("Data Bronze Path Loaded: %s", paths.data_bronze)
    logger.info("Data Bronze Training Path Loaded: %s", paths.data_bronze_train)
    logger.info("Data Bronze Testing Path Loaded: %s", paths.data_bronze_test)



In [4]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [5]:
# Get Path's Object
paths = get_paths()

# Create bronze log path 
bronze_log_path = paths.logs / "bronze.log"

# Initial Logger
capstone_logger = configure_logging(
    "capstone",
    bronze_log_path,
    level=logging.DEBUG,
    overwrite_handlers=True,
)

# Initiate Logger and log file
logger = logging.getLogger("capstone.bronze")

# Log load and initiation
logger.info("Bronze stage starting")

log_bronze_paths(paths, logger)


2026-01-04 04:17:52,790 | INFO | capstone.bronze | Bronze stage starting
2026-01-04 04:17:52,792 | INFO | capstone.bronze | Project Root Path Loaded: /workspace
2026-01-04 04:17:52,794 | INFO | capstone.bronze | Project Logging Path Loaded: /workspace/logs
2026-01-04 04:17:52,796 | INFO | capstone.bronze | Project Artifacts Path Loaded: /workspace/artifacts
2026-01-04 04:17:52,797 | INFO | capstone.bronze | Project Notebooks Path Loaded /workspace/notebooks
2026-01-04 04:17:52,799 | INFO | capstone.bronze | Project Raw Path Loaded: /workspace/data/raw
2026-01-04 04:17:52,801 | INFO | capstone.bronze | Project Data Path Loaded: /workspace/data
2026-01-04 04:17:52,803 | INFO | capstone.bronze | Data Bronze Path Loaded: /workspace/data/bronze
2026-01-04 04:17:52,804 | INFO | capstone.bronze | Data Bronze Training Path Loaded: /workspace/data/bronze/train
2026-01-04 04:17:52,805 | INFO | capstone.bronze | Data Bronze Testing Path Loaded: /workspace/data/bronze/test


In [6]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [7]:
# Paths
RAW_DATA_PATH = paths.data_raw
BRONZE_DATA_PATH = paths.data_bronze_train

BRONZE_ARTIFACTS_PATH = paths.artifacts / "bronze"
LOGS_PATH = paths.logs

# Make sure Paths Exist
BRONZE_DATA_PATH.mkdir(parents=True, exist_ok=True)
BRONZE_ARTIFACTS_PATH.mkdir(parents=True, exist_ok=True)
LOGS_PATH.mkdir(parents=True, exist_ok=True)

# META Details
LAYER_NAME = "bronze"
BRONZE_VERSION = "bronze__001"

# W&B
WANDB_PROJECT = "capstone"
WANDB_ENTITY = "dcoo230-western-governors-university"
WANDB_RUN_NAME = f"{BRONZE_VERSION}"

# Raw File Details
RAW_FILE_PATH = paths.data_raw / "pump_sensor_data"
RAW_FILE_NAME = "sensor.csv"

# Dataset Details
DATASET_NAME = "PUMP"
SPLIT_STATUS = "unsplit"
LABEL_TYPE = None
LABEL_SOURCE = None
RUN_ID = "run__001"
ASSET_ID = "asset__001"

ADD_RECORD_ID = True
RECORD_ID_INPUTS = ["meta__source_file", "meta__source_row_id"]
RECORD_ID_METHOD = "hash_pandas_object_uint64"

# Dataframe Friendly
LABEL_TYPE_DF = pd.NA if LABEL_TYPE is None else LABEL_TYPE
LABEL_SOURCE_DF = pd.NA if LABEL_SOURCE is None else LABEL_SOURCE


In [8]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [9]:
wandb_run = wandb.init(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    name=WANDB_RUN_NAME,
    job_type="bronze",
    config={
        "dataset_name": DATASET_NAME,
        "bronze_version": BRONZE_VERSION,
        "raw_data_file": RAW_FILE_NAME,
        "raw_path": str(RAW_FILE_PATH / RAW_FILE_NAME),
        "split_status": SPLIT_STATUS,
        "label_type": LABEL_TYPE,
        "label_source": LABEL_SOURCE,
        "run_id": RUN_ID,
        "asset_id": ASSET_ID,
        
        "add_record_id": ADD_RECORD_ID,
        "record_id_inputs": RECORD_ID_INPUTS if ADD_RECORD_ID else None,
        "record_id_method": RECORD_ID_METHOD if ADD_RECORD_ID else None,
        
        "bronze_out_path": str(BRONZE_DATA_PATH),
    },
)

logger.info("W&B initialized: %s", wandb_run.name)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dcoo230 (dcoo230-western-governors-university). Use `wandb login --relogin` to force relogin


2026-01-04 04:17:56,546 | INFO | capstone.bronze | W&B initialized: bronze__001


In [10]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [11]:
# Load Data

pump_df = ingest_data(
    RAW_FILE_PATH,
    file_name=RAW_FILE_NAME,
    dataset_name=DATASET_NAME,
    split=SPLIT_STATUS,
    label_type=LABEL_TYPE_DF,
    label_source=LABEL_SOURCE_DF,
    run_id=RUN_ID,
    asset_id=ASSET_ID,
    add_record_id=ADD_RECORD_ID,
    validate=True
)

2026-01-04 04:17:57,137 | INFO | capstone.file_io | Loading Data file: /workspace/data/raw/pump_sensor_data/sensor.csv
2026-01-04 04:18:04,004 | INFO | capstone.file_io | Loaded Data File: sensor.csv | shape=(220320, 55) | columns=['Unnamed: 0', 'timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40', 'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45', 'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'sensor_50', 'sensor_51', 'machine_status']


In [12]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [13]:
# Basic Dataframe Information/Summary

# Shape 
print("Shape:", pump_df.shape)
logger.info("Shape: %s", pump_df.shape)

# Dtypes as a compact block
print("\nData types:")
print(pump_df.dtypes)
logger.info("Dtypes:\n%s", pump_df.dtypes.to_string())

# Memory Usages
print("\nMemory usage (MB):")
print(pump_df.memory_usage(deep=True).sum() / (1024 ** 2))
mem_mb = pump_df.memory_usage(deep=True).sum() / (1024 ** 2)
logger.info("Memory usage (MB): %.2f", mem_mb)

# Head(15) as text (truncate columns for readability)
print("\nFirst 15 rows:")
display(pump_df.head(15))
logger.info("Head(15):\n%s", pump_df.head(15).to_string(max_cols=40, max_rows=15))

# Describe Numbers
print("\nBasic numeric summary:")
display(pump_df.describe(include=[np.number]).T)
desc_num = pump_df.describe(include=[np.number]).T
logger.info("Numeric describe (truncated):\n%s", desc_num.to_string(max_rows=60))

# Describe Objects and categorical
print("\nBasic object / categorical summary:")
object_category_columns = pump_df.select_dtypes(include=["object", "category", "string"]).columns
if len(object_category_columns):
    desc_obj = pump_df[object_category_columns].describe().T
    display(desc_obj)
    logger.info("Object/categorical describe (truncated):\n%s", desc_obj.to_string(max_rows=60))
else:
    logger.info("No object/category/string columns to describe.")

# Meta Columns
print("\nMeta Columns Summary:")
meta_columns = [column for column in pump_df.columns if column.startswith("meta__")]
pump_df[meta_columns].head(3)
logger.info("Meta Columns: (%d): %s", len(meta_columns), "\n".join(meta_columns))

# All Other Columns
print("\nAll Other Columns Summary:")
other_columns = [column for column in pump_df.columns if not column.startswith("meta__")]
pump_df[other_columns].head(3)
logger.info("All Other Columns: (%d): %s", len(other_columns), "\n".join(other_columns))

# Missing
missing = (pump_df[other_columns].isna().mean() * 100).sort_values(ascending=False).head(20)
display(missing.to_frame("pct_missing"))
logger.info("Top missingness (%%):\n%s", missing.to_string())


2026-01-04 04:18:05,095 | INFO | capstone.bronze | Shape: (220320, 65)
2026-01-04 04:18:05,105 | INFO | capstone.bronze | Dtypes:
meta__dataset                       category
meta__split                         category
meta__run_id                          object
meta__asset_id                        object
meta__label_type                    category
meta__label_source                  category
meta__ingested_at_utc    datetime64[us, UTC]
meta__source_file             string[python]
meta__source_row_id                    int64
meta__record_id                       uint64
Unnamed: 0                             int64
timestamp                             object
sensor_00                            float64
sensor_01                            float64
sensor_02                            float64
sensor_03                            float64
sensor_04                            float64
sensor_05                            float64
sensor_06                            float64
sensor_07      

Shape: (220320, 65)

Data types:
meta__dataset       category
meta__split         category
meta__run_id          object
meta__asset_id        object
meta__label_type    category
                      ...   
sensor_48            float64
sensor_49            float64
sensor_50            float64
sensor_51            float64
machine_status        object
Length: 65, dtype: object

Memory usage (MB):
166.04556369781494


2026-01-04 04:18:05,463 | INFO | capstone.bronze | Memory usage (MB): 166.05



First 15 rows:


,meta__dataset,meta__split,meta__run_id,meta__asset_id,meta__label_type,meta__label_source,meta__ingested_at_utc,meta__source_file,meta__source_row_id,meta__record_id,Unnamed: 0,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,0,14598431322315673869,0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,1,15954729095895098000,1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,NaN,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,2,10041703297090838359,2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,NaN,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,3,2072036352569063261,3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,NaN,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,4,4365040424004714369,4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,NaN,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL
5,PUMP,unsplit,run__001,asset__001,NaN,NaN,2026-01-04 04:18:04.015707+00:00,sensor.csv,5,13731207034657248969,5,2018-04-01 00:05:00,2.453588,47.09201,53.1684,46.397568,637.6157,78.18568,13.41146,16.16753,15.89265,15.16204,39.29406,49.37051,32.2381

...

,count,unique,top,freq
meta__dataset,220320,1,PUMP,220320
meta__split,220320,1,unsplit,220320
meta__run_id,220320,1,run__001,220320
meta__asset_id,220320,1,asset__001,220320
meta__label_type,0,0,NaN,NaN
meta__label_source,0,0,NaN,NaN
meta__source_file,220320,1,sensor.csv,220320
timestamp,220320,220320,2018-08-31 23:43:00,1
machine_status,220320,3,NORMAL,205836


2026-01-04 04:18:06,947 | INFO | capstone.bronze | Object/categorical describe (truncated):
                     count  unique                  top    freq
meta__dataset       220320       1                 PUMP  220320
meta__split         220320       1              unsplit  220320
meta__run_id        220320       1             run__001  220320
meta__asset_id      220320       1           asset__001  220320
meta__label_type         0       0                  NaN     NaN
meta__label_source       0       0                  NaN     NaN
meta__source_file   220320       1           sensor.csv  220320
timestamp           220320  220320  2018-08-31 23:43:00       1
machine_status      220320       3               NORMAL  205836
2026-01-04 04:18:06,968 | INFO | capstone.bronze | Meta Columns: (10): meta__dataset
meta__split
meta__run_id
meta__asset_id
meta__label_type
meta__label_source
meta__ingested_at_utc
meta__source_file
meta__source_row_id
meta__record_id
2026-01-04 04:18:07,022 | INFO 


Meta Columns Summary:

All Other Columns Summary:


,pct_missing
sensor_15,100.000000
sensor_50,34.956881
sensor_51,6.982117
sensor_00,4.633261
sensor_07,2.474129
sensor_08,2.317992
sensor_06,2.177741
sensor_09,2.085603
sensor_01,0.167484
sensor_30,0.118464


2026-01-04 04:18:07,095 | INFO | capstone.bronze | Top missingness (%):
sensor_15    100.000000
sensor_50     34.956881
sensor_51      6.982117
sensor_00      4.633261
sensor_07      2.474129
sensor_08      2.317992
sensor_06      2.177741
sensor_09      2.085603
sensor_01      0.167484
sensor_30      0.118464
sensor_29      0.032680
sensor_32      0.030864
sensor_18      0.020879
sensor_17      0.020879
sensor_22      0.018609
sensor_25      0.016340
sensor_16      0.014070
sensor_45      0.012255
sensor_39      0.012255
sensor_43      0.012255


In [14]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [15]:
# Parquet DType Stablity 
# Casting my two pd.NA columns to strings to prevent against NullType failures

pump_df["meta__label_type"] = pump_df["meta__label_type"].astype("string")

pump_df["meta__label_source"] = pump_df["meta__label_source"].astype("string")

In [16]:
# Save Data as Parquet
save_data(pump_df, paths.data_bronze_train, "pump__bronze__train")

2026-01-04 04:18:08,047 | INFO | capstone.file_io | Saving DataFrame to Parquet: /workspace/data/bronze/train/pump__bronze__train.parquet
2026-01-04 04:18:09,762 | INFO | capstone.file_io | Saved: pump__bronze__train.parquet | shape=(220320, 65) | columns=['meta__dataset', 'meta__split', 'meta__run_id', 'meta__asset_id', 'meta__label_type', 'meta__label_source', 'meta__ingested_at_utc', 'meta__source_file', 'meta__source_row_id', 'meta__record_id', 'Unnamed: 0', 'timestamp', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 

PosixPath('/workspace/data/bronze/train/pump__bronze__train.parquet')

In [17]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [18]:
metrics, saved = profile_dataframe(pump_df, logger, artifacts_dir=paths.root/"artifacts"/"bronze")



2026-01-04 04:18:10,578 | INFO | capstone.bronze | Shape: (220320, 65)
2026-01-04 04:18:10,580 | INFO | capstone.bronze | Memory (MB): 189.16
2026-01-04 04:18:10,583 | INFO | capstone.bronze | Dtypes:
meta__dataset                       category
meta__split                         category
meta__run_id                          object
meta__asset_id                        object
meta__label_type              string[python]
meta__label_source            string[python]
meta__ingested_at_utc    datetime64[us, UTC]
meta__source_file             string[python]
meta__source_row_id                    int64
meta__record_id                       uint64
Unnamed: 0                             int64
timestamp                             object
sensor_00                            float64
sensor_01                            float64
sensor_02                            float64
sensor_03                            float64
sensor_04                            float64
sensor_05                         

In [ ]:
finalize_wandb_stage(
    wandb_run,
    stage="bronze",
    dataframe=pump_df,
    project_root=paths.root,
    logs_dir=paths.logs,
    dataset_dirs=[paths.data_bronze_train],
    dataset_artifact_name="pump-bronze-train-parquet",
    notebook_path=paths.notebooks / "Bronze_Preprocessing_Pump.ipynb",
)

TypeError: finalize_wandb_stage() got an unexpected keyword argument 'df'

In [ ]:
wandb_run.finish()

cols,▁
memory_mb,▁
rows,▁
cols,65
memory_mb,166.04556
rows,220320


In [ ]:
'''

# Ensure artifacts subdir exists
bronze_art_dir = paths.artifacts / "bronze"
bronze_art_dir.mkdir(parents=True, exist_ok=True)

# Profile + export describe() CSVs into artifacts/bronze
metrics, saved = profile_dataframe(pump_df, logger, artifacts_dir=bronze_art_dir, head=15)

# Start or reuse a W&B run (you can also init at the top; either is fine)
run = wandb.init(project="capstone", job_type="bronze", config={"dataset": "pump", "stage": "bronze"})

# Log scalars + preview table near the end
log_metrics(run, metrics)
log_dataframe_head(run, pump_df, key="bronze_head15", n=15)

# Upload logs (just bronze.log)
log_files_as_artifact(
    run,
    artifact_name="capstone-logs-bronze",
    artifact_type="logs",
    files=[paths.logs / "bronze.log"],
    aliases=["latest"],
    metadata={"stage": "bronze", "dataset": "pump"},
)

# Upload bronze parquet outputs (train dir)
log_dir_as_artifact(
    run,
    artifact_name="pump-bronze-train-parquet",
    artifact_type="dataset",
    dir_path=paths.data_bronze_train,
    patterns=("*.parquet", "*.pq"),  # explicit
    aliases=["latest"],
    metadata={"stage": "bronze", "split": "train", "dataset": "pump"},
)

# Upload bronze diagnostics exports (artifacts/bronze)
log_dir_as_artifact(
    run,
    artifact_name="pump-bronze-diagnostics",
    artifact_type="eda",
    dir_path=bronze_art_dir,
    patterns=("*.csv", "*.json", "*.log", "*.txt"),
    aliases=["latest"],
    metadata={"stage": "bronze", "dataset": "pump"},
)

# Upload the notebook itself (point this at where you actually save it)
# Best practice: save/copy the notebook into paths.notebooks first.
bronze_nb = paths.notebooks / "Bronze_Preprocessing_Pump.ipynb"

if bronze_nb.exists():
    log_files_as_artifact(
        run,
        artifact_name="capstone-notebooks",
        artifact_type="notebook",
        files=[bronze_nb],
        aliases=["latest"],
        metadata={"stage": "bronze"},
    )
else:
    logger.warning("Notebook not found at %s; skipping upload.", bronze_nb)

run.finish()

'''


'\n\n# Ensure artifacts subdir exists\nbronze_art_dir = paths.artifacts / "bronze"\nbronze_art_dir.mkdir(parents=True, exist_ok=True)\n\n# Profile + export describe() CSVs into artifacts/bronze\nmetrics, saved = profile_dataframe(pump_df, logger, artifacts_dir=bronze_art_dir, head=15)\n\n# Start or reuse a W&B run (you can also init at the top; either is fine)\nrun = wandb.init(project="capstone", job_type="bronze", config={"dataset": "pump", "stage": "bronze"})\n\n# Log scalars + preview table near the end\nlog_metrics(run, metrics)\nlog_dataframe_head(run, pump_df, key="bronze_head15", n=15)\n\n# Upload logs (just bronze.log)\nlog_files_as_artifact(\n    run,\n    artifact_name="capstone-logs-bronze",\n    artifact_type="logs",\n    files=[paths.logs / "bronze.log"],\n    aliases=["latest"],\n    metadata={"stage": "bronze", "dataset": "pump"},\n)\n\n# Upload bronze parquet outputs (train dir)\nlog_dir_as_artifact(\n    run,\n    artifact_name="pump-bronze-train-parquet",\n    artifa

In [ ]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 